In [1]:
from pyhealth.datasets import UMLSDataset

umls_ds = UMLSDataset(
    root="https://storage.googleapis.com/pyhealth/umls/",
    dev=True,
)

/home/pj20/miniconda3/envs/pyhealth/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
umls_ds.stat()


Statistics of base dataset (dev=True):
	- Dataset: UMLSDataset
	- Number of triples: 88176
	- Number of entities: 9737
	- Number of relations: 8
	- Task name: Null
	- Number of samples: 0



In [3]:
from pyhealth.tasks import link_prediction_fn

umls_ds = umls_ds.set_task(link_prediction_fn, negative_sampling=256)

Loading UMLSDataset base dataset from /home/pj20/.cache/pyhealth/datasets/46e7370273967c215741135e6ccdd2b9.pkl


In [4]:
umls_ds.stat()


Statistics of base dataset (dev=True):
	- Dataset: UMLSDataset
	- Number of triples: 88176
	- Number of entities: 9737
	- Number of relations: 8
	- Task name: link_prediction_fn
	- Number of samples: 176352



In [5]:
from pyhealth.datasets import split_by_keys, get_dataloader_kg

train_dataset, val_dataset, test_dataset = split_by_keys(umls_ds, [0.8, 0.1, 0.1])

In [6]:
len(train_dataset['head'])

70540

In [7]:
batch_size = 256

train_loader, num_batch_train = get_dataloader_kg(train_dataset, batch_size=batch_size, shuffle=True)
val_loader, num_batch_val = get_dataloader_kg(val_dataset, batch_size=batch_size, shuffle=False)
test_loader, num_batch_test = get_dataloader_kg(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
from pyhealth.models.kg import TransE, RotatE

model = RotatE(
    dataset=umls_ds
)

In [9]:
from pyhealth.trainer import Trainer
import torch

trainer = Trainer(model=model, device='cuda:6')
trainer.train(
    train_dataloader=train_loader,
    epochs=20,
    batch_size=batch_size,
    ## num_batch=num_batch_train,
    num_batch=100,
    optimizer_params={
        'lr': 1e-3
    }
)

RotatE()
Metrics: None
Device: cuda:6

Training:
Batch size: 256
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.001}
Weight decay: 0.0
Max grad norm: None
Val dataloader: None
Monitor: None
Monitor criterion: max
Epochs: 20

Epoch 0 / 20: 100%|██████████| 100/100 [00:01<00:00, 79.10it/s]
--- Train epoch-0, step-100 ---
loss: 1.5616

Epoch 1 / 20: 100%|██████████| 100/100 [00:01<00:00, 81.11it/s]
--- Train epoch-1, step-200 ---
loss: 0.8167

Epoch 2 / 20: 100%|██████████| 100/100 [00:01<00:00, 78.95it/s]
--- Train epoch-2, step-300 ---
loss: 0.6330

Epoch 3 / 20: 100%|██████████| 100/100 [00:01<00:00, 79.72it/s]
--- Train epoch-3, step-400 ---
loss: 0.4502

Epoch 4 / 20: 100%|██████████| 100/100 [00:01<00:00, 81.36it/s]
--- Train epoch-4, step-500 ---
loss: 0.2979

Epoch 5 / 20: 100%|██████████| 100/100 [00:01<00:00, 84.04it/s]
--- Train epoch-5, step-600 ---
loss: 0.1957

Epoch 6 / 20: 100%|██████████| 100/100 [00:01<00:00, 90.28it/s]
--- Train epoch-6, step-700 

KeyboardInterrupt: 